# 3I025 &mdash; Mini-projet : Cooperative Path-Finding

Ariana CARNIELLI et David HERZOG

Dans ce projet on a mis en œuvre quelques algorithmes pour la recherche coopérative de chemins dans une carte. L’idée est de trouver une façon de bouger des personnages vers des objectifs sans qu’il y ait des collisions entre eux, ce qui peut être intéressant pour le codage des jeux par exemple. La base de ces algorithmes est l’algorithme A* et chacun d’entre eux implémente des techniques supplémentaires différentes pour empêcher des collisions.

Dans le modèle utilisé chaque joueur a connaissance de la carte et aussi de la position de tous les autres joueurs. Il connaît aussi le chemin choisi par les autres, alors on peut voir le modèle comme un jeu à information parfaite.  

Nous avons codé trois algorithmes de recherche coopérative : 

- Une stratégie opportuniste qui recalcule des parties de chemins quand elle détecte une collision dans quelques pas, dite Slicing. Chaque chemin est donc calculé dans un premier temps sans se soucier des chemins (et positions) des autres ;


- Une stratégie coopérative de base qui essaie d’exécuter la plus grande quantité possible de chemins en parallèle, en considérant une collision comme un croisement de chemins sans prendre en compte le temps ;


- Une stratégie coopérative plus avancée qui calcule les chemins des joueurs en considérant à quel pas de temps ils passeront sur une case donnée. Nous avons fait deux versions de cette stratégie, une dans laquelle le chemin de chaque joueur est calculé jusqu’à son objectif et une deuxième où le chemin est recalculée à chaque `d` pas de temps. 

Nous avons crée six cartes supplémentaires (numerotées de 5 à 10) aux cartes données en exemple pour montrer les caractéristiques de chacune de ces stratégies. En plus nous avons repris quatre cartes codées par d'autres groupes (numerotées de 11 à 14). Chaque stratégie est implémentée dans une classe propre et notre code contient aussi une classe pour le calcul de l’heuristique utilisé par les deux dernières stratégies. En plus, on a deux autres fichiers, `utils.py` et `main.py`, avec des fonctions simples utilisés par plusieurs stratégies, une exception personnalisée, et un point d’entrée pour les tests qui est aussi utilisé dans ce notebook. 

La suite de ce notebook présente chacune des stratégies de façon plus détaillée, en expliquant nos choix d’implémentation et les difficultés rencontrées avec aussi quelques exemples.  

In [1]:
import main

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


## Stratégie de slicing

Dans cette stratégie on commence par calculer un chemin jusqu’à la cible pour chaque joueur sans prendre en considération les chemins des autres. Cela est fait par une implémentation de l’algorithme A\* en utilisant la distance de Manhattan comme heuristique.  Il s’agit d’une heuristique admissible pour ce problème donc on sait que A\* retourne bien une solution optimale. Comme elle est aussi consistante on sait que pour chaque point qui est sorti du tas, sa distance est bien optimale et il ne rentrera plus jamais dans le tas, alors on peut faire une sortie anticipé de la méthode des qu’on retire la position de la cible du tas.  

Une fois les chemins calculés, on les exécute de façon parallèle. Tous les `n` pas de temps, l’algorithme regarde, pour chaque chemin, `m` pas de temps dans le futur pour voir s’il y aura une collision. Si c’est le cas, il recalcule le morceau de chemin entre les positions actuelle et dans `m` pas et remplace dans le chemin les `m` pas suivants par ce nouveau morceau. S’il n’arrive pas à trouver un tel chemin ou si le chemin trouvé est trop long (plus grande ou égale à `max_slice*m`), on recalcule toute la trajectoire jusqu’à la cible. Si il ne trouve toujours pas un chemin possible, le joueur reste immobile pour les `n` prochains pas. On fait en sorte qu’un joueur qui doit rester immobile est considéré comme un obstacle par les autres joueurs, ce qui empêche de collisions.

Les valeurs de `m`, `n` et `max_slice` sont passées en argument au constructeur de la classe `Slicing` qui implémente l’algorithme. On doit avoir `m > n` si on veut être certain que tous les morceaux des chemins seront analysés pour éviter de collisions. 

Cette stratégie donne des très bons résultats dans plusieurs cas mais elle ne permet pas d’éviter certaines situations de blocage : Par exemple, si deux joueurs veulent traverser un tunnel plus long que `m` dans de sens opposés, ils finissent par rester immobiles. Les cellules suivantes donnent les exécutions de chacune des cartes avec cette stratégie :

In [ ]:
p = main.Projet(boardNumber = 1, fps = 5, iterations = 50)
p.mainSlicing(goalStates = [(12, 6), (19, 8), (6, 7)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 2, fps = 5, iterations = 50)
p.mainSlicing(goalStates = [(9, 13), (4, 9), (12, 6)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 3, fps = 5, iterations = 50)
p.mainSlicing(goalStates = [(9, 13), (4, 9), (12, 6)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 4, fps = 5, iterations = 50)
p.mainSlicing(goalStates = [(4, 9), (9, 13), (12, 6), (12, 7), (13, 6), (13, 7)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 5, fps = 5, iterations = 50)
p.mainSlicing(goalStates = [(19, 17), (19, 18), (19, 19), (0, 17), (0, 18), (0, 19)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 6, fps = 5, iterations = 50)
p.mainSlicing(goalStates = [(0, 9), (0, 8), (0, 7)], verbose = False)

Le prochain exemple montre une situation de blocage : 

In [ ]:
p = main.Projet(boardNumber = 7, fps = 5, iterations = 25)
p.mainSlicing(goalStates = [(0, 9), (0, 1)], verbose = False)

Le blocage est levé si on augmente la valeur de `m` :

In [ ]:
p = main.Projet(boardNumber = 7, fps = 5, iterations = 50)
p.mainSlicing(goalStates = [(0, 9), (0, 1)], m = 7, verbose = False)

In [ ]:
p = main.Projet(boardNumber = 8, fps = 5, iterations = 50)
p.mainSlicing(goalStates = [(0, 9), (0, 1)], verbose = False)

Le prochain exemple montre une situation de blocage qui ne peut pas être résolue avec une augmentation de `m` : 

In [ ]:
p = main.Projet(boardNumber = 9, fps = 5, iterations = 25)
p.mainSlicing(goalStates = [(9, 9), (4, 7)], verbose = False)

Cet exemple produit une blocage car les cibles sont sur le chemin unique possible pour un joueur :

In [ ]:
p = main.Projet(boardNumber = 10, fps = 5, iterations = 50)
p.mainSlicing(goalStates = [(8, 8), (8, 11)], verbose = False)

Ici un joueur est obligé de recalculer sont chemin plusieurs fois ce qui augmente la quantité nécessaire de pas :

In [ ]:
p = main.Projet(boardNumber = 11, fps = 5, iterations = 50)
p.mainSlicing(goalStates = [(2, 8), (4, 4), (3, 6), (5, 10)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 12, fps = 5, iterations = 50)
p.mainSlicing(goalStates = [(3, 3), (8, 7), (3, 6)], verbose = False)

Ici aussi on a une blocage puisque les joueurs se bloquent un à l'autre :

In [ ]:
p = main.Projet(boardNumber = 13, fps = 5, iterations = 25)
p.mainSlicing(goalStates = [(5, 2), (0, 0)], verbose = False)

Cette dernière carte est impossible aussi (en effet elle est impossible pour tous les algorithmes que nous avons codé) :

In [ ]:
p = main.Projet(boardNumber = 14, fps = 5, iterations = 25)
p.mainSlicing(goalStates = [(5, 2), (1, 2)], verbose = False)

## Stratégie coopérative de base

Cette deuxième stratégie essaie de prendre en considération dans l’exécution d’un chemin des positions des autres joueurs. Cela est fait en prenant les chemins des autres joueurs comme s’ils étaient des obstacles et n’en jouant en parallèle que les joueurs qui n’ont pas de collisions entre eux. Cela produit des résultats positifs mais généralement très longs par rapport à la première stratégie. Pour cette stratégie, `self.iterations` est le nombre maximal d'itérations **par groupe**. Le nombre maximal d'itérations peut être au pire cas égal à nbPlayers * self.iterations.

Plus précisément, on commence par calculer un chemin pour chaque joueur avec l’algorithme A\* en considérant les positions initiales et les cibles des autres joueurs comme des obstacles pour éviter de collisions. Ces chemins ne seront pas modifiés et l’algorithme cherchera un ordre d’exécution sans collisions. Pour ce faire, il décompose l’ensemble des chemins en groupes qui peuvent être exécutés en parallèle. D’abord on calcule la matrice de collision, une matrice carré de taille `n` où `n` est la quantité de joueurs, telle que la case`[i, j]` contient 1 s'il y a de collisions entre les chemins `i` et` j`, 0 sinon et -1 si `i = j`. On utilise cette matrice pour déterminer quel est le chemin avec le moins de collisions et ce chemin est pris comme premier chemin du premier groupe. Ce groupe commence en ne contenant que ce chemin et ensuite d'autres chemins sont ajoutés successivement si ils ne croisent pas les chemins déjà présents.  On répète cette construction jusqu’à ce que tous les chemins soient dans un groupe et les groupes sont exécutés séquentiellement.

Les collisions sont détectés de façon naïve : deux chemins sont en collision si ils ont au moins une position en commun, sans prendre en compte les instants où les joueurs passent par cette position. La décomposition en groupe décrite précédemment n’est pas optimale. Étant donné les longueurs des chemins et leurs croisements, le problème de trouver la décomposition en groupes optimale est relié au problème de trouver des cliques dans un graphe (les sommets sont les chemins et deux sommets sont reliés par une arête si ils n’ont pas de collisions), qui est NP-hard. 

In [ ]:
p = main.Projet(boardNumber = 1, fps = 5, iterations = 100)
p.mainCoopBase([(12, 6), (19, 8), (6, 7)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 2, fps = 5, iterations = 50)
p.mainCoopBase(goalStates = [(9, 13), (4, 9), (12, 6)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 3, fps = 5, iterations = 50)
p.mainCoopBase(goalStates = [(9, 13), (4, 9), (12, 6)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 4, fps = 5, iterations = 50)
p.mainCoopBase(goalStates = [(4, 9), (9, 13), (12, 6), (12, 7), (13, 6), (13, 7)], verbose = False)

Dans les trois prochains cas touts les chemins se croissent entre eux et alors ils sont exécutés de façon séquentiel :

In [ ]:
p = main.Projet(boardNumber = 5, fps = 5, iterations = 50)
p.mainCoopBase(goalStates = [(19, 17), (19, 18), (19, 19), (0, 17), (0, 18), (0, 19)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 6, fps = 5, iterations = 50)
p.mainCoopBase(goalStates = [(0, 9), (0, 8), (0, 7)], verbose = False)

Ce cas était un blocage pour le `Slicing` mais fonctionne avec cette stratégie : 

In [ ]:
p = main.Projet(boardNumber = 7, fps = 5, iterations = 25)
p.mainCoopBase(goalStates = [(0, 9), (0, 1)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 8, fps = 5, iterations = 50)
p.mainCoopBase(goalStates = [(0, 9), (0, 1)], verbose = False)

Les deux prochains restent encore impossibles :

In [ ]:
p = main.Projet(boardNumber = 9, fps = 5, iterations = 25)
p.mainCoopBase(goalStates = [(9, 9), (4, 7)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 10, fps = 5, iterations = 25)
p.mainCoopBase(goalStates = [(8, 8), (8, 11)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 11, fps = 5, iterations = 50)
p.mainCoopBase(goalStates = [(2, 8), (4, 4), (3, 6), (5, 10)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 12, fps = 5, iterations = 50)
p.mainCoopBase(goalStates = [(3, 3), (8, 7), (3, 6)], verbose = False)

Ce cas reste impossible aussi :

In [ ]:
p = main.Projet(boardNumber = 13, fps = 5, iterations = 25)
p.mainCoopBase(goalStates = [(5, 2), (0, 0)], verbose = False)

## Stratégie coopérative avancée

La stratégie coopérative de base permet d’éviter des collisions mais comme elle ne prend pas en considération le temps les solutions données sont souvent très longues. Pour l’améliorer on prend en  considération le temps en suivant les idées données dans [cet article](http://www0.cs.ucl.ac.uk/staff/d.silver/web/Publications_files/coop-path-AIWisdom.pdf) . L’idée principale est de considérer une dimension supplémentaire pour le temps, ce que l’on fait en implémentant le *Tableau de réservation* de l’article. Nos obstacles sont ainsi décomposés en deux : les obstacles fixes qui ne possèdent pas une coordonnée de temps et les obstacles mobiles (les autres joueurs).

Chaque joueur calcule son chemin plus court à sa cible en utilisant l’algorithme A\* adapté pour prendre en compte la dimension temporelle. Le premier joueur n’a aucun obstacle mobile, le deuxième prend en compte le chemin du premier et ainsi de suite. Alors, on a une priorité naturelle du premier joueur. Parfois cela cause de blocages qu’on a cherché à résoudre avec l’implémentation décrite a la prochaine section.    

Comme mentionné dans l’article, pour éviter des calculs trop longs, on utilise comme heuristique la distance réelle, obtenue par une application de l’algorithme A\* en espace avec uniquement les obstacles fixes et dans le sens inversé, en partant de la cible et cherchant le chemin le plus court vers la position initiale du joueur. Cette implémentation de l’algorithme A\* pour le calcul de la distance réelle a été faite dans une classe séparée, `algo_A`, car elle a quelques spécificités. En particulier, l’initialisation et la boucle principale sont faites dans des fonctions séparées et la boucle principale ne tourne que jusqu’à ce qu’on trouve la distance à la case voulue.


In [ ]:
p = main.Projet(boardNumber = 1, fps = 5, iterations = 50)
p.mainTempA([(12, 6), (19, 8), (6, 7)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 2, fps = 5, iterations = 50)
p.mainTempA(goalStates = [(9, 13), (4, 9), (12, 6)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 3, fps = 5, iterations = 50)
p.mainTempA(goalStates = [(9, 13), (4, 9), (12, 6)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 4, fps = 5, iterations = 50)
p.mainTempA(goalStates = [(4, 9), (9, 13), (12, 6), (12, 7), (13, 6), (13, 7)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 5, fps = 5, iterations = 50)
p.mainTempA(goalStates = [(19, 17), (19, 18), (19, 19), (0, 17), (0, 18), (0, 19)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 6, fps = 5, iterations = 50)
p.mainTempA(goalStates = [(0, 9), (0, 8), (0, 7)], verbose = False)

Ces deux prochains cas ont un comportement très intéressant :

In [ ]:
p = main.Projet(boardNumber = 7, fps = 5, iterations = 50)
p.mainTempA(goalStates = [(0, 9), (0, 1)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 8, fps = 5, iterations = 50)
p.mainTempA(goalStates = [(0, 9), (0, 1)], verbose = False)

Ce cas était insoluble avant mais cette stratégie arrive à le résoudre (mais seulement car le joueur rouge a la priorité) : 

In [ ]:
p = main.Projet(boardNumber = 9, fps = 5, iterations = 25)
p.mainTempA(goalStates = [(9, 9), (4, 7)], verbose = False)

L’exemple suivant reste insoluble mais ne produit plus d’erreur :

In [ ]:
p = main.Projet(boardNumber = 10, fps = 5, iterations = 50)
p.mainTempA(goalStates = [(8, 8), (8, 11)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 11, fps = 5, iterations = 50)
p.mainTempA(goalStates = [(2, 8), (4, 4), (3, 6), (5, 10)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 12, fps = 5, iterations = 50)
p.mainTempA(goalStates = [(3, 3), (8, 7), (3, 6)], verbose = False)

Ce cas peut désormais être résolu :

In [ ]:
p = main.Projet(boardNumber = 13, fps = 5, iterations = 25)
p.mainTempA(goalStates = [(5, 2), (0, 0)], verbose = False)

## Stratégie coopérative avancée avec profondeur de recherche fixée

Suivant toujours des idées de l’article mentionné précédemment on modifie la stratégie de coopération temporelle pour arrêter la recherche au bout d’un nombre fixe `d` de pas. Cela a quelques avantages : 

- La priorité n’est plus fixé par l’ordre de joueurs mais change au cours du temps ;

- On n’a pas besoin de calculer toutes les trajectoires jusqu’à leurs cibles d’un seul coup, ce qui réduit l’espace mémoire utilisé ;

- Elle permet de débloquer quelques autres situations où l’objectif d’un des joueurs est dans le chemin d’un autre.

Cependant, il y a encore de cas qui ne sont pas traitables.

Pour implémenter cette méthode, on crée un tableau supplémentaire pour stocker les prochaines étapes de la trajectoire des joueurs. À chaque pas de temps, si la case correspondant à un joueur est vide, on calcule les `d` prochains pas de sa trajectoire à l’aide d’un algorithme A\* modifié pour s’arrêter dès que la première case contenant un temps `t = t0 + d` est trouvée, où `t0` est l’instant où cet algorithme est appelé. Pour éviter que les joueurs fassent leurs recalculs au même moment (ce qui revient à la stratégie précédente) on utilise des valeurs différentes de `d` pour chaque jouer à la première fois qu’on calcule des chemins. 

On fait aussi un traitement du cas où un joueur n’arrive pas à trouver un chemin valable lorsqu’il recalcule sa trajectoire : on efface la trajectoire future de tous les autres joueurs  et on les recalcule en commençant par le joueur qui a déclenché le problème pour lui donner la priorité maximale. 

In [ ]:
p = main.Projet(boardNumber = 1, fps = 5, iterations = 50)
p.mainTempA_D([(12, 6), (19, 8), (6, 7)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 2, fps = 5, iterations = 50)
p.mainTempA_D(goalStates = [(9, 13), (4, 9), (12, 6)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 3, fps = 5, iterations = 50)
p.mainTempA_D(goalStates = [(9, 13), (4, 9), (12, 6)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 4, fps = 5, iterations = 50)
p.mainTempA_D(goalStates = [(4, 9), (9, 13), (12, 6), (12, 7), (13, 6), (13, 7)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 5, fps = 5, iterations = 50)
p.mainTempA_D(goalStates = [(19, 17), (19, 18), (19, 19), (0, 17), (0, 18), (0, 19)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 6, fps = 5, iterations = 50)
p.mainTempA_D(goalStates = [(0, 9), (0, 8), (0, 7)], verbose = False)

Les deux prochains cas ont un comportament pas optimal en comparaison à la stratégie précédente :

In [ ]:
p = main.Projet(boardNumber = 7, fps = 5, iterations = 50)
p.mainTempA_D(goalStates = [(0, 9), (0, 1)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 8, fps = 5, iterations = 50)
p.mainTempA_D(goalStates = [(0, 9), (0, 1)], verbose = False)

Ils peuvent être améliorés en changeant le paramètre `d` :

In [ ]:
p = main.Projet(boardNumber = 7, fps = 5, iterations = 50)
p.mainTempA_D(goalStates = [(0, 9), (0, 1)], d = 11, verbose = False)

In [ ]:
p = main.Projet(boardNumber = 8, fps = 5, iterations = 50)
p.mainTempA_D(goalStates = [(0, 9), (0, 1)], d = 11, verbose = False)

In [ ]:
p = main.Projet(boardNumber = 9, fps = 5, iterations = 25)
p.mainTempA_D(goalStates = [(9, 9), (4, 7)], d = 10, verbose = False)

Ce cas est finalement résoluble grâce à la priorité tournante : 

In [ ]:
p = main.Projet(boardNumber = 10, fps = 5, iterations = 50)
p.mainTempA_D(goalStates = [(8, 8), (8, 11)], verbose = False)

Cette prochaine carte était possible avant mais ne l’est plus avec le paramètre `d`= 7.  Cela démontre que l’algorithme n’est pas toujours la meilleur choix entre les algorithmes codés.

In [ ]:
p = main.Projet(boardNumber = 11, fps = 5, iterations = 50)
p.mainTempA_D(goalStates = [(2, 8), (4, 4), (3, 6), (5, 10)], verbose = False)

Néanmoins, en changeant `d`, on arrive à la résoudre avec cet algorithme :

In [ ]:
p = main.Projet(boardNumber = 11, fps = 5, iterations = 50)
p.mainTempA_D(goalStates = [(2, 8), (4, 4), (3, 6), (5, 10)], d = 15, verbose = False)

In [ ]:
p = main.Projet(boardNumber = 12, fps = 5, iterations = 50)
p.mainTempA_D(goalStates = [(3, 3), (8, 7), (3, 6)], verbose = False)

In [ ]:
p = main.Projet(boardNumber = 13, fps = 5, iterations = 25)
p.mainTempA_D(goalStates = [(5, 2), (0, 0)], verbose = False)

## Comparaison des stratégies

Le tableau suivant donne le nombre de pas nécessaire pour que les joueurs atteignent leurs cibles :

carte | Slicing | Coop Base | Coop Avancé | Coop Avancé D
------|---------|-----------|-------------|----------------
1 | 34 | 53 | 30 | 29
2 | 22 | 22 | 22 | 21
3 | 16 | 25 | 16 | 15
4 | 23 | 28 | 14 | 13
5 | 50 | 216 | 41 | 40
6 | 33 | 69 | 27 | 25
7 | 27 (m = 7) | 28 | 23 | 22 (d = 11)
8 | 29 | 28 | 23 | 22 (d = 11)
9 | block | erreur | 20 | 20 (d = 11)
10 | block | erreur | block | 25 
11 | 33 | 32 | 17 | 16 (d = 15)
12 | 10 | 30 | 10 | 9
13 | block | erreur | 12 | 11

Les paramètres utilisés sont ceux par défaut sauf lorsque cela est indiqué en parenthèse. 

On remarque que la stratégie coopérative de base a toujours les pires résultats parce que plusieurs joueurs passent trop longtemps arrêtés. Les deux stratégies avancées améliorent beaucoup par rapport aux autres et permettent de débloquer certains cas. Néanmoins elles ne sont pas infaillibles : la carte 14 reste insoluble. 

Nous n’avons pas comparé les temps de calcul car les techniques utilisés sont très diverses. En particulier, certaines calculent les chemins avant le début de l’exécution dans un seul bloc alors que d’autres font des calculs tout au cours de l’exécution. Ainsi c’est très compliqué de mesurer le temps de calcul sans prendre en compte l’exécution. On observe néanmoins que toutes les méthodes sont très rapides. 

Les seules méthodes lentes sont celles des stratégies coopératives avancées en utilisant la distance de Manhattan comme heuristique à la place de la distance réelle, ce qui était attendu, et alors on n’a pas gardé ces implémentations dans la version finale. 